In [ ]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!kaggle datasets download -d chiragsoni/ferdata
!unzip ferdata.zip

In [ ]:
train_data_dir = 'train/'
valid_data_dir = 'test/'

In [ ]:
class FacialEmotionDataset(Dataset):
    def __init__(self, folder):
        angry = glob(folder + 'angry/*.jpg') 
        disgust = glob(folder + 'disgust/*.jpg') 
        fear = glob(folder + 'fear/*.jpg')
        happy = glob(folder + 'happy/*.jpg')
        neutral = glob(folder + 'neutral/*.jpg')
        sad = glob(folder + 'sad/*.jpg')
        surprise = glob(folder + 'surprise/*.jpg')
        self.fpaths = angry + disgust + fear + happy + neutral + sad + surprise

        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        from random import shuffle, seed; seed(10); shuffle(self.fpaths)
        self.idx2emotion = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
        self.emotion2idx = {emotion:ix for ix, emotion in enumerate(self.idx2emotion)}
        self.targets = [self.emotion2idx[path_name.split('/')[-2]] for path_name in self.fpaths]
    def __len__(self): return len(self.fpaths)
    def __getitem__(self, ix):
        f = self.fpaths[ix]
        target = self.targets[ix]
        im = (cv2.imread(f)[:,:,::-1])
        im = cv2.resize(im, (48,48))
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im = torch.tensor(im/255)
        im = im.permute(2,0,1)
        im = self.normalize(im) 
        return im.float().to(device), torch.tensor(target).to(device)

        

In [ ]:
data = FacialEmotionDataset(train_data_dir)

In [ ]:
idx = np.random.randint(0, 100)
im, label = data[idx]
plt.imshow(im.permute(1,2,0).cpu().numpy())

In [ ]:
def get_model():
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    for param in model.layer4.parameters():
        param.requires_grad = True
    model.fc = nn.Sequential(
        nn.Linear(512, 7),
    )

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)
    return model.to(device), loss_fn, optimizer
model, loss_fn, optimizer = get_model()
print(model)

In [ ]:
!pip install torch_summary
from torchsummary import summary
model, criterion, optimizer = get_model()
summary(model, torch.zeros(1,3,48,48))

In [ ]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [ ]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

@torch.no_grad()
def val_loss(x, y, model, loss_fn):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [ ]:
def get_data():
    train = FacialEmotionDataset(train_data_dir)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True, drop_last = True)
    val = FacialEmotionDataset(valid_data_dir)
    val_dl = DataLoader(val, batch_size=32, shuffle=True, drop_last = True)
    return trn_dl, val_dl

In [ ]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()

In [ ]:
from tqdm import tqdm

num_epochs = 10

train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
train_losses, train_accuracies = [], []
for epoch in range(num_epochs):
  train_epoch_losses, train_epoch_accuracies = [], []
  for ix, batch in enumerate(iter(trn_dl)):
    x, y = batch
    batch_loss = train_batch(x, y, model, optimizer, loss_fn)
    train_epoch_losses.append(batch_loss)        
  train_epoch_loss = np.array(train_epoch_losses).mean()

  for ix, batch in enumerate(iter(trn_dl)):
    x, y = batch
    is_correct = accuracy(x, y, model)
    train_epoch_accuracies.extend(is_correct)
  train_epoch_accuracy = np.mean(train_epoch_accuracies)

  # validation loss
  # model.eval()
  val_epoch_losses, val_epoch_accuracies = [], []
  for ix, batch in enumerate(iter(val_dl)):
    x, y = batch
    val_batch_loss = val_loss(x, y, model, loss_fn)
    val_is_correct = accuracy(x, y, model)

    val_epoch_losses.append(val_batch_loss)
    val_epoch_accuracies.extend(val_is_correct)
  val_epoch_loss = np.mean(val_epoch_losses)
  val_epoch_accuracy = np.mean(val_epoch_accuracies)
  # scheduler.step(val_epoch_loss)

  # append to losses and accuracies
  train_losses.append(train_epoch_loss)
  train_accuracies.append(train_epoch_accuracy)
  val_losses.append(val_epoch_loss)
  val_accuracies.append(val_epoch_accuracy)
  print(f"epoch {epoch + 1}/{num_epochs}, train_epoch_loss:{train_epoch_loss}, train_epoch_accuracy: {train_epoch_accuracy}, val_epoch_accuracy: {val_epoch_accuracy}")

print(f'Min training loss: {min(train_losses):.4f}')
print(f'Max training accuracy: {max(train_accuracies):.4f}')
print(f'Min validation loss: {min(val_losses):.4f}')
print(f'Max validation accuracy: {max(val_accuracies):.4f}')

In [ ]:
# plot accuracies
plt.plot(train_accuracies, label='Training accuracy', color='blue', linestyle='--')
plt.plot(val_accuracies, label='Validation accuracy', color='red')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy value over increasing epochs')

plt.tight_layout()
plt.show()


In [ ]:
print(train_accuracies)
print(val_accuracies)